# Churn Telco
---

# Análisis de Churn de Clientes en Telecomunicaciones
## Framework Completo de Análisis

---

## **Abstract**

El churn de clientes representa uno de los mayores desafíos para las empresas de telecomunicaciones, con costos de adquisición de nuevos clientes que superan 5-25 veces el costo de retener clientes existentes. Este análisis desarrollará un sistema predictivo de propensión al churn utilizando técnicas de machine learning sobre datos transaccionales y demográficos de clientes.

*Objetivo*
* Construccion de un ML de clasificacion binaria
* Objetivizar la propensión a la conversión
* Validar cuales son las caracteristicas mas influyentes en este modelo

**Metodología**: Análisis exploratorio de datos, ingeniería de features, modelado predictivo con algoritmos de clasificación, y validación del modelo mediante métricas de negocio.

**Impacto Esperado**: Reducción del 15-25% en la tasa de churn mediante identificación temprana y estrategias de retención dirigidas.

---

## **Hipótesis de Análisis**

### **H1: Hipótesis de Lealtad Temporal**
- **Hipótesis**: Los clientes con mayor tenure (antigüedad) tienen menor propensión al churn
- **Justificación**: La inversión de tiempo y la familiaridad con el servicio crean barreras de salida
- **Test**: Correlación negativa entre tenure y churn rate

### **H2: Hipótesis de Compromiso Contractual**
- **Hipótesis**: Los contratos de largo plazo (2 años) reducen significativamente el churn vs. month-to-month
- **Justificación**: Los contratos largos implican penalizaciones por cancelación anticipada
- **Test**: Comparación de churn rates por tipo de contrato

### **H3: Hipótesis de Valor Percibido**
- **Hipótesis**: Existe una relación U-invertida entre MonthlyCharges y churn (churn alto en extremos de precio)
- **Justificación**: Clientes con precios muy bajos pueden tener servicios limitados; precios muy altos generan sensibilidad al costo
- **Test**: Análisis de churn rate por quintiles de precio

### **H4: Hipótesis de Servicios Adicionales**
- **Hipótesis**: Los clientes con servicios de valor agregado (OnlineSecurity, TechSupport) tienen menor churn
- **Justificación**: Más servicios incrementan el switching cost y la dependencia
- **Test**: Churn rate por número de servicios adicionales contratados

### **H5: Hipótesis Demográfica**
- **Hipótesis**: Los adultos mayores sin dependientes tienen mayor propensión al churn
- **Justificación**: Menor tolerancia a la tecnología y menos necesidades de conectividad familiar
- **Test**: Segmentación por SeniorCitizen + Dependents

### **H6: Hipótesis de Método de Pago**
- **Hipótesis**: Los métodos de pago automáticos reducen el churn vs. pagos manuales
- **Justificación**: Mayor fricción en pagos manuales puede generar insatisfacción
- **Test**: Churn rate por PaymentMethod

---

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# setting display
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
def pandas_sqlite_read(db_path, query, params=None):
    with sqlite3.connect(db_path) as conn:
        # Leer directamente a DataFrame
        if params:
            df = pd.read_sql_query(query, conn, params=params)
        else:
            df = pd.read_sql_query(query, conn)
    
    return df

In [ ]:
class ChurnDatabase:
    
    def __init__(self, db_path):
        self.db_path = db_path
        
    def get_all_customers(self):
        query = "SELECT * FROM telco_customer_churn"
        return pandas_sqlite_read(self.db_path, query)
    
    def get_churn_customers(self):
        query = "SELECT * FROM telco_customer_churn WHERE Churn = 'Yes'"
        return pandas_sqlite_read(self.db_path, query)
    
    def get_customers_by_contract(self, contract_type):
        query = "SELECT * FROM telco_customer_churn WHERE Contract = ?"
        return pandas_sqlite_read(self.db_path, query, params=[contract_type])
    
    def get_high_value_customers(self, min_charges):
        query = """
        SELECT customerID, gender, tenure, Contract, 
               MonthlyCharges, TotalCharges, Churn
        FROM telco_customer_churn
        WHERE TotalCharges > ?
        ORDER BY TotalCharges DESC
        """
        return pandas_sqlite_read(self.db_path, query, params=[min_charges])
    
    def get_churn_analysis_data(self):
        
        query = """
        SELECT 
            gender,
            SeniorCitizen,
            Partner,
            Dependents,
            tenure,
            PhoneService,
            MultipleLines,
            InternetService,
            OnlineSecurity,
            OnlineBackup,
            DeviceProtection,
            TechSupport,
            StreamingTV,
            StreamingMovies,
            Contract,
            PaperlessBilling,
            PaymentMethod,
            MonthlyCharges,
            TotalCharges,
            Churn,
            -- Features derivadas
            CASE 
                WHEN tenure <= 12 THEN 'New'
                WHEN tenure <= 36 THEN 'Regular' 
                ELSE 'Loyal'
            END as customer_segment,
            
            CASE 
                WHEN MonthlyCharges < 35 THEN 'Low'
                WHEN MonthlyCharges < 65 THEN 'Medium'
                ELSE 'High'
            END as price_segment,
            
            -- Número de servicios adicionales
            (CASE WHEN OnlineSecurity = 'Yes' THEN 1 ELSE 0 END +
             CASE WHEN OnlineBackup = 'Yes' THEN 1 ELSE 0 END +
             CASE WHEN DeviceProtection = 'Yes' THEN 1 ELSE 0 END +
             CASE WHEN TechSupport = 'Yes' THEN 1 ELSE 0 END +
             CASE WHEN StreamingTV = 'Yes' THEN 1 ELSE 0 END +
             CASE WHEN StreamingMovies = 'Yes' THEN 1 ELSE 0 END) as additional_services
             
        FROM telco_customer_churn
        """
        return pandas_sqlite_read(self.db_path, query)
    
    def get_churn_summary_stats(self):

        query = """
        SELECT 
            Contract,
            PaymentMethod,
            COUNT(*) as total_customers,
            SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) as churn_customers,
            ROUND(
                100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 
                2
            ) as churn_rate,
            ROUND(AVG(MonthlyCharges), 2) as avg_monthly_charges,
            ROUND(AVG(tenure), 2) as avg_tenure
        FROM telco_customer_churn
        GROUP BY Contract, PaymentMethod
        ORDER BY churn_rate DESC
        """
        return pandas_sqlite_read(self.db_path, query)
    
    def get_dataframe_by_query(self, query:str):
        return pandas_sqlite_read(self.db_path, query)

In [ ]:
churn = ChurnDatabase("../database/telco_customer_churn.sqlite.db")

In [ ]:
churn_summary = churn.get_churn_summary_stats()

In [ ]:
churn_summary

In [ ]:
def create_churn_heatmap(df):
    """Heatmap mostrando churn rate por Contract vs PaymentMethod"""
    
    # Crear pivot table
    pivot_data = df.pivot(index='Contract', columns='PaymentMethod', values='churn_rate')
    
    plt.figure(figsize=(16, 9))
    
    # Crear heatmap
    sns.heatmap(pivot_data, 
                annot=True, 
                cmap='Reds', 
                fmt='.1f',
                cbar_kws={'label': 'Churn Rate (%)'},
                linewidths=0.5)
    
    plt.title('CHURN RATE HEATMAP\nContract Type vs Payment Method', 
              fontsize=16, fontweight='bold', pad=20)
    
    plt.xlabel('Payment Method', fontsize=12, fontweight='bold')
    plt.ylabel('Contract Type', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Añadir insights como texto
    # plt.figtext(0.02, 0.02, 
    #             "Key Insight: Month-to-month + Electronic check = Highest churn (53.7%)\n"
    #             "Best retention: Two year + Mailed check (0.8% churn)", 
    #             fontsize=10, style='italic', 
    #             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
    
    plt.show()


In [ ]:
create_churn_heatmap(churn_summary)

In [ ]:
def create_bubble_chart(df):
    """Bubble chart: Churn Rate vs Monthly Charges, size = Customer Volume"""
    
    plt.figure(figsize=(16, 9))
    
    # Crear colores por tipo de contrato
    colors = {'Month-to-month': '#FF6B6B', 'One year': '#4ECDC4', 'Two year': '#45B7D1'}
    
    for contract in df['Contract'].unique():
        contract_data = df[df['Contract'] == contract]
        
        plt.scatter(contract_data['avg_monthly_charges'], 
                   contract_data['churn_rate'],
                   s=contract_data['total_customers'] / 5,  # Tamaño proporcional
                   alpha=0.7,
                   c=colors[contract],
                   label=contract,
                   edgecolors='black',
                   linewidth=1)
        
        # Añadir etiquetas para puntos críticos
        for idx, row in contract_data.iterrows():
            if row['churn_rate'] > 30 or row['churn_rate'] < 5:
                plt.annotate(f"{row['PaymentMethod']}\n({row['churn_rate']:.1f}%)", 
                           (row['avg_monthly_charges'], row['churn_rate']),
                           xytext=(5, 5), textcoords='offset points',
                           fontsize=8, ha='left')
    
    plt.xlabel('Average Monthly Charges ($)', fontsize=12, fontweight='bold')
    plt.ylabel('Churn Rate (%)', fontsize=12, fontweight='bold')
    plt.title('CHURN vs PRICING ANALYSIS\n(Bubble size = Customer Volume)', 
              fontsize=16, fontweight='bold', pad=20)
              
    plt.legend(title='Contract Type', title_fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # Añadir líneas de referencia
    plt.axhline(y=df['churn_rate'].mean(), color='red', linestyle='--', alpha=0.5, 
                label=f'Avg Churn ({df["churn_rate"].mean():.1f}%)')
    
    plt.tight_layout()
    plt.show()


In [ ]:
create_bubble_chart(churn_summary)

In [ ]:
df = churn.get_churn_analysis_data()

# EDA Exploracion inicial
---

In [ ]:
df.info()

In [ ]:
df.duplicated().any()

In [ ]:
df.isnull().sum()

In [ ]:
for col in df.columns:
    print(f"Column: {col}")
    print(f"Num unique: {df[col].nunique()}")
    print("Unique values:", df[col].unique())
    print("-" * 40)



In [ ]:
# Validar que no tenga columna con un valor unico
for col in df.columns:
    if len(df[col].unique())==1:
        print(f"{col}has only one unique value:{df[col].unique()}")

In [ ]:

mask_empty_all = df.applymap(lambda x: isinstance(x, str) and x.strip() == '')

# Count of empty strings per column
empty_count_per_col = mask_empty_all.sum()

print("Number of empty strings per column:")
print(empty_count_per_col[empty_count_per_col != 0])

In [ ]:
df[df["TotalCharges"] == ""]

### Insights de la data source: 
- 7_043 rows , 21 cols originales, agregadas algunas carcateristicas que pueden ayudar a dar algunas caracteristicas mas consistentes que otros.
- Se debe cambiar algunos datos, ajustandolos de object a float
- No hay data sucia necesariamente pero hay algunos datos en `()` y algunos con valores '' en TotalCharges
- No hay data duplicada


---

In [ ]:
df["PaymentMethod"] = df.PaymentMethod.str.replace(" (automatic)","")
df.PaymentMethod.unique()

In [ ]:
def ratio_check(serie_column: pd.Series, tope_value:float = 0.5) -> bool:

    ratio = serie_column.isna().sum() / serie_column.count()
    if ratio <= tope_value:

        transform_ratio_perc = ratio * 100.00
        print(f"El ratio: {transform_ratio_perc:.4f}, si es menor a 10% se puede imputar, si es mayor a 40% se puede retirar la columna")
    else:
        print("Cumple con caracteristicas ratio minimo")

In [ ]:
#limpieza de la variable TotalCharges

df["TotalCharges"]= df["TotalCharges"].replace(" ",np.nan).replace("", np.nan)
#Convertir a valor flotante
df["TotalCharges"]= df["TotalCharges"].astype(float)
#Fill NaN con la median, pues el ratio de na/total_amount es minimo

ratio_check(df["TotalCharges"])

df["TotalCharges"]= df["TotalCharges"].fillna(df["TotalCharges"].median())


## Analisis UniVariado

**Variable Target -> Churn**

In [ ]:
## Distribucion del Dato Churn
churn_counts = df['Churn'].value_counts()

# Colors mapping
colors = {'Yes': '#874A4A', 'No': '#9EBC8A'}

# Create subplot (1 row, 2 columns)
fig, axes = plt.subplots(1, 2, figsize=(16, 9))

# Pie chart 
axes[0].pie(
    churn_counts,
    labels=churn_counts.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=[colors[label] for label in churn_counts.index]
)

# axes[0].set_title('Distribucion de clase de Churn', fontsize=14)

# Bar chart 
bars = axes[1].bar(
    churn_counts.index,
    churn_counts.values,
    color=[colors[label] for label in churn_counts.index]
)

# Add numeric labels on top of bars
for bar in bars:
    height = bar.get_height()
    axes[1].text(
        bar.get_x() + bar.get_width()/2, 
        height + (0.01 * max(churn_counts.values)),  # a bit above the bar
        f'{int(height)}', 
        ha='center', va='bottom', fontsize=10
    )

axes[1].set_title('Distribucion de clase de Churn', fontsize=14)
axes[1].set_ylabel('Count')
axes[1].set_xlabel('Churn')

# Adjust layout
fig.suptitle("CHurn vs Distribucion Poblacional")
plt.tight_layout()
plt.show()



*Claramente observamos un desbalance de la cantidad de datos en la variable predictoria*

---

- *Analizar la relacion de las variables del tipo categoria vs target, para entender como influye cada una de las caracteristicas sobre el modelo.*
- *Las variables numericas y sus tendencias nos data mayor inferencia en como se correlacionan respecto a la variable target*
- *Tambien recordar como afecta la colinealidad y la multicolinealidad con la performance del analisis y de la respuesta en el modelo*

In [ ]:
import math

cat_cols = [col for col in df.columns if df[col].nunique() <= 10 and col != "Churn"]

n_cols = 3  # 3 graphics by row
n_rows = math.ceil(len(cat_cols) / n_cols)

# Create figure
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4))
axes = axes.flatten()  

for i, col in enumerate(cat_cols):
    sns.countplot(x=col, hue="Churn", data=df, ax=axes[i])
    axes[i].set_title(f"{col} vs Churn")

for j in range(len(cat_cols), len(axes)):
    fig.delaxes(axes[j])


plt.tight_layout()
plt.show()

In [ ]:
def pie_chart(df, columns, autopct='%1.1f%%'):
    # Define custom palette
    custom_palette = ['#73946B', '#9EBC8A', '#DDEB9D', '#F8ED8C', '#F9C784', '#FCAF58', '#F98948']
    
    num_cols = 3  # number of charts per row
    num_rows = int(np.ceil(len(columns) / num_cols))
    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))
    axes = axes.flatten()
    
    for i, column in enumerate(columns):
        value_counts = df[column].value_counts()
        
        # Repeat colors if number of categories exceeds palette length
        colors = (custom_palette * ((len(value_counts) // len(custom_palette)) + 1))[:len(value_counts)]
        
        axes[i].pie(
            value_counts,
            labels=value_counts.index,
            autopct=autopct,
            startangle=90,
            colors=colors
        )
        axes[i].set_title(f"Distribution of {column}", fontsize=12)
        axes[i].axis('equal')
    
    # Remove unused axes
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Detalle de coqueteria que me parecio super interesante: (me gusta mucho mas esta opcion por ser de pocas variables)
pie_chart(df=df, columns=cat_cols)

Que es lo que podemos observar en base a lo que nos otorgan estos graficos?

-   Contract Type -> Hay una gran relacion con el churn, por lo tanto los clientes con un contrato del tipo `month-to-month` tienen un rate de churn mucho mas alto a comparacion del resto.
-   Payment method: Los que tienen pagos via electronic tienden a tener un churn mayor comparandolo con el resto
-   Internet Service - (Such as OnlineSecurity, TechSupport, and DeviceProtection) , tienen una correlacion con el churn, los que tienen esos servicios tienen esa tendencia al churn.
-   PaperlessBilling -> Posiblemente correlacionado por el metodo de pago que realiza al ser pago online
Observando lo demas podemos incidir que el resto de variables no tienen mucho peso en el comportamiento del cliente, tomando un papel secundario en la inferencia de las predicciones.


---
## Analisis biVariado

In [ ]:
cat_cols_bivariate = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 
                      'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 
                      'Contract', 'PaperlessBilling', 'PaymentMethod']

def barchart_stacked(df, features, feature1='Churn'):
    colors = {'Yes': '#ce5454', 'No': '#9EBC8A'}

    ncols = 2
    nrows = int(np.ceil(len(features) / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 5*nrows))
    axes = axes.flatten()

    for i, feature2 in enumerate(features):
        ax = axes[i]

        # Prepare stacked data
        data = df.groupby([feature2, feature1]).size().reset_index(name='count')
        pivot_data = data.pivot(index=feature2, columns=feature1, values='count').fillna(0)

        # Plot stacked bars
        bottom_vals = np.zeros(len(pivot_data))
        for churn_status in colors.keys():
            ax.bar(
                pivot_data.index,
                pivot_data[churn_status],
                bottom=bottom_vals,
                color=colors[churn_status],
                label=churn_status
            )
            # Add number inside each segment
            for idx, val in enumerate(pivot_data[churn_status]):
                if val > 0:
                    ax.text(
                        idx, 
                        bottom_vals[idx] + val/2,
                        f"{int(val)}",
                        ha='center', va='center',
                        color='white', fontsize=10
                    )
            bottom_vals += pivot_data[churn_status]

        # Add total number on top
        for idx, total in enumerate(bottom_vals):
            ax.text(
                idx,
                total + (max(bottom_vals) * 0.02),
                f"{int(total)}",
                ha='center', va='bottom',
                fontsize=10
            )

        ax.set_title(f'{feature2} vs {feature1}')
        ax.set_xlabel(feature2)
        ax.set_ylabel('Number of Customers')
        ax.set_ylim(0, max(bottom_vals) * 1.15)

        # Rotate x-axis labels
        ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='right')

        # Move legend outside plot
        ax.legend(title=feature1, bbox_to_anchor=(1.05, 1), loc='upper left')

    # Remove empty axes
    for j in range(len(features), len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

In [ ]:
barchart_stacked(df, cat_cols_bivariate)

In [ ]:
def barchart_bivariate(df, features, feature1='Churn'):
    colors = {'Yes': '#ce5454', 'No': '#9EBC8A'}
    
    ncols = 2
    nrows = int(np.ceil(len(features) / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 5*nrows))
    axes = axes.flatten()

    for i, feature2 in enumerate(features):
        ax = axes[i]
        
        # Calculate counts
        data = df.groupby([feature2, feature1]).size().reset_index(name='count')

        # Create barplot
        sns.barplot(
            data=data,
            x=feature2,
            y='count',
            hue=feature1,
            palette=colors,
            ax=ax
        )

        # Add numeric labels on top of bars
        for p in ax.patches:
            height = p.get_height()
            ax.annotate(
                f"{int(height)}",
                (p.get_x() + p.get_width()/2., height),
                ha='center', va='bottom',
                xytext=(0, 6),  # distance from top of bar
                textcoords='offset points',
                color='black', fontsize=11
            )

        ax.set_title(f'{feature2} vs {feature1}')
        ax.set_xlabel(feature2)
        ax.set_ylabel('Number of Customers')
        ax.set_ylim(0, max(data['count']) * 1.15)

        # Rotate x-axis labels
        ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='right')

        # Move legend outside the plot
        ax.legend(title=feature1, bbox_to_anchor=(1.05, 1), loc='upper left')

    # Remove empty axes if number of features is less than the grid slots
    for j in range(len(features), len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

In [ ]:
barchart_bivariate(df, cat_cols_bivariate)


Estos son los principales factores que influyen en la tasa de churn:

### Demografía del cliente

* **Género:** Los clientes masculinos y femeninos tienen tasas de deserción casi idénticas, por lo que el género no es un factor determinante. (dependiendo de la correlacion podriamos hasta removerlo de las caracteristicas dentro del modelo)

* **Tercera edad:** Los clientes de la tercera edad tienen una tasa de deserción más alta en comparación con los que no lo son. Esto puede deberse a la sensibilidad a los precios o a la necesidad de servicios especiales. -> Una variable bastante interesante.

* **Pareja y dependientes:** Los clientes que no tienen pareja ni dependientes son más propensos al churn, ya que suelen ser más "móviles" a la hora de cambiar de servicio. -> Muy buena variable

---

### Detalles del servicio

* **Servicio de internet:** Los clientes con servicio de **fibra óptica** tienen la mayor tasa de deserción, seguidos por los de **DSL**. Los clientes que no tienen servicio de internet presentan la tasa más baja. Esto probablemente esté influenciado por las expectativas de precio y rendimiento.

* **Servicios adicionales:** La falta de servicios adicionales (como seguridad en línea, respaldo, soporte técnico, etc.) se relaciona con una mayor deserción, lo que presenta una oportunidad para ventas adicionales.
* **Contrato:** Los contratos **mes a mes** tienen una tasa de deserción significativamente más alta que los contratos a 1 o 2 años. Los contratos a largo plazo son una manera efectiva de reducir la deserción.

---

### Facturación y pago
* **Facturación sin papel:** Los clientes que optan por la facturación sin papel tienen una tasa de deserción más alta, lo que a menudo está relacionado con contratos flexibles mes a mes.
* **Método de pago:** Los clientes que pagan con **cheque electrónico** presentan la tasa de deserción más alta. Los pagos por transferencia bancaria, tarjeta de crédito y cheque por correo tienen una tasa de deserción más baja. El cheque electrónico es un método de pago común entre los clientes de alto riesgo.

In [ ]:
def displot(df, features, feature1='Churn'):
    n_features = len(features)
    ncols = 1
    nrows = (n_features + 1) // 1
    
    plt.figure(figsize=(16, 5*nrows))
    
    # Warna untuk kelas churn
    palette = {
        'Yes': '#ce5454',    # Not Churn
        'No' : '#9EBC8A'    # Churn
    }
    
    for i, feature2 in enumerate(features, 1):
        plt.subplot(nrows, ncols, i)
        
        for cls in df[feature1].unique():
            subset = df[df[feature1] == cls]
            sns.kdeplot(subset[feature2], 
                        shade=True, 
                        alpha=0.5, 
                        color=palette[cls], 
                        label='Churn' if cls == 'Yes' else 'Not Churn')
        
        plt.title(f'{feature2} vs {feature1}')
        plt.xlabel(feature2)
        plt.ylabel('Density')
        plt.legend()
    
    plt.tight_layout()
    plt.show()


In [ ]:
displot(df, num_cols:= ['tenure', 'MonthlyCharges', 'TotalCharges'])

### Perfil de los clientes con alta deserción
* **Contrato y facturación:** Los clientes con contrato **mes a mes** y que usan **facturación sin papel** tienen un riesgo de deserción mucho mayor. Esto a menudo se relaciona con la flexibilidad de su servicio.

* **Servicio de internet:** La **fibra óptica** tiene la tasa de deserción más alta, seguida por **DSL**. La falta de servicios adicionales (como seguridad en línea, respaldo, o soporte técnico) también está asociada con una mayor deserción.

* **Demografía y pagos:** Los clientes **sin pareja ni dependientes** y los **adultos mayores** son más propensos a desertar. El pago con **cheque electrónico** es el método con la tasa de deserción más alta.

* **Cargos mensuales:** Existe un pico de deserción en el rango de $70 a $100 en cargos mensuales, lo que sugiere una sensibilidad al precio en este segmento.

---

### Estrategias de retención
* **Enfoque en clientes nuevos:** La mayoría de los clientes que se van tienen una baja antigüedad y cargos totales bajos. Es crucial enfocar los esfuerzos de retención en la experiencia inicial del cliente, especialmente durante los **primeros meses**.

* **Fidelización:** Los clientes con mayor antigüedad y cargos totales son mucho menos propensos a irse. La tasa de deserción es muy baja después de 20 meses.

* **Oportunidades:** Se recomienda apuntar a los clientes nuevos y a aquellos con facturas altas, ofreciéndoles promociones, un mejor proceso de incorporación y servicios de valor agregado para reducir la deserción.

* **Contratos a largo plazo:** Fomentar contratos a 1 o 2 años es una manera efectiva de reducir la deserción, ya que los contratos a largo plazo demuestran ser un factor clave de retención.

## Correlacion con HEATMAP

In [ ]:
def create_churn_focused_heatmap(df_encoded, target_variable='Churn'):
    """Heatmap enfocado en correlaciones con la variable de churn"""
    
    # Calcular correlación
    corr_matrix = df_encoded.corr()
    
    # Obtener correlaciones con la variable target
    if target_variable in corr_matrix.columns:
        target_correlations = corr_matrix[target_variable].abs().sort_values(ascending=False)
        
        # Seleccionar top variables más correlacionadas con churn
        top_variables = target_correlations.head(15).index.tolist()
        
        # Crear submatriz con las variables más relevantes
        corr_subset = corr_matrix.loc[top_variables, top_variables]
    else:
        corr_subset = corr_matrix
    
    # Crear heatmap
    fig = go.Figure(data=go.Heatmap(
        z=corr_subset.values,
        x=corr_subset.columns,
        y=corr_subset.index,
        colorscale='RdBu_r',  # Rojo-Azul invertido
        zmid=0,
        text=np.round(corr_subset.values, 3),
        texttemplate="%{text}",
        textfont={"size": 10},
        colorbar=dict(
            title="Correlation with<br>Churn Variable",
            # titlefont=dict(size=12)
        )
    ))
    
    # Destacar la fila/columna del target
    if target_variable in corr_subset.columns:
        target_idx = corr_subset.columns.get_loc(target_variable)
        
        # Añadir líneas para destacar la variable target
        fig.add_hline(y=target_idx, line_width=3, line_color="red", opacity=0.7)
        fig.add_vline(x=target_idx, line_width=3, line_color="red", opacity=0.7)
    
    fig.update_layout(
        title={
            'text': f'🎯 Churn-Focused Correlation Analysis<br><sub>Top variables correlated with {target_variable}</sub>',
            'x': 0.5,
            'font': {'size': 18}
        },
        width=900,
        height=700,
        xaxis=dict(tickangle=45),
        yaxis=dict(autorange='reversed')
    )
    
    fig.show()
    
    # Mostrar ranking de correlaciones con churn
    if target_variable in df_encoded.columns:
        churn_corr = df_encoded.corr()[target_variable].abs().sort_values(ascending=False)
        print(f"\n🔍 Top 15 variables correlated with {target_variable}:")
        print("=" * 50)
        for var, corr in churn_corr.head(15).items():
            if var != target_variable:
                print(f"{var:<25}: {corr:.3f}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, FunctionTransformer, StandardScaler

def create_preprocessing_pipeline(X: pd.DataFrame) -> Pipeline:
    # --- Columnas ---
    bin_cols = ["gender","Partner","Dependents","PhoneService","PaperlessBilling","Churn"]
    ordinal_cols = ["customer_segment", "price_segment", "additional_services"]
    num_cols = ["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]
    multi_cols_for_dummies = [
        "MultipleLines","InternetService","OnlineSecurity","OnlineBackup",
        "DeviceProtection","TechSupport","StreamingTV","StreamingMovies",
        "Contract","PaymentMethod"
    ]
    
    # --- Mapas y categorías ---
    mapper_columns_bins = {"Yes":1, "No":0, "Male":1, "Female":0}
    ordinal_categories = [
        ["New", "Regular", "Loyal"],  # customer_segment
        ["Low", "Medium", "High"],    # price_segment
        [0, 1, 2, 3, 4, 5, 6]        # additional_services
    ]
    
    def map_bins(X): 
        return X.applymap(lambda x: mapper_columns_bins.get(x, x))
    
    def to_numeric(X):
        """Convierte todas las columnas a tipo numérico"""
        if isinstance(X, np.ndarray):
            # Si es numpy array, convertir a DataFrame temporalmente
            df = pd.DataFrame(X)
            return df.apply(pd.to_numeric, errors='coerce').fillna(0).values
        else:
            # Si es DataFrame
            return X.apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # --- Transformadores ---
    bin_transformer = FunctionTransformer(func=map_bins, validate=False)
    ordinal_transformer = OrdinalEncoder(categories=ordinal_categories)
    multi_transformer = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    scaling = StandardScaler()
    numeric_transformer = FunctionTransformer(func=to_numeric, validate=False)
    
    # --- Filtramos solo columnas presentes ---
    bin_cols_present = [c for c in bin_cols if c in X.columns]
    ordinal_cols_present = [c for c in ordinal_cols if c in X.columns]
    multi_cols_present = [c for c in multi_cols_for_dummies if c in X.columns]
    num_cols_present = [c for c in num_cols if c in X.columns]
    
    # --- ColumnTransformer ---
    preprocessor = ColumnTransformer(
        transformers=[
            ("bin", bin_transformer, bin_cols_present),
            ("ord", ordinal_transformer, ordinal_cols_present),
            ("multi", multi_transformer, multi_cols_present),
            ("scale", scaling, num_cols_present)
        ],
        remainder="passthrough"
    )
    
    # --- Pipeline con conversión a numéricos ---
    pipe = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("to_numeric", numeric_transformer)
    ])
    
    return pipe

### Datos completamente numericos para analisis del tipo de correlacion

In [ ]:
def reconstruct_transformed_df(pipe: Pipeline, df:pd.DataFrame, target_col:str="Churn"):
    """
    Reconstruye un DataFrame transformado desde un pipeline entrenado, 
    preservando nombres de columnas para cada tipo de transformador.
    """
    preprocessor_fitted = pipe.named_steps["preprocessor"]
    
    # Columnas originales
    bin_cols = ["gender","Partner","Dependents","PhoneService","PaperlessBilling","Churn"]
    ordinal_cols = ["customer_segment", "price_segment", "additional_services"]
    num_cols = ["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]
    multi_cols_for_dummies = [
        "MultipleLines","InternetService","OnlineSecurity","OnlineBackup",
        "DeviceProtection","TechSupport","StreamingTV","StreamingMovies",
        "Contract","PaymentMethod"
    ]
    
    # --- Transformamos los datos ---
    df_trans = pipe.fit_transform(df)
    
    if hasattr(df_trans, "toarray"):
        df_trans = df_trans.toarray()
    
    # --- Columnas por tipo de transformador ---
    bin_features = [c for c in bin_cols if c in df.columns]
    ord_features = [c for c in ordinal_cols if c in df.columns]
    
    multi_features = preprocessor_fitted.named_transformers_["multi"].get_feature_names_out(multi_cols_for_dummies)
    
    scaling_features = [c for c in num_cols if c in df.columns]
    
    remainder_features = [
        col for col in df.columns
        if col not in bin_features + ord_features + multi_cols_for_dummies + scaling_features
    ]
    
    # --- Concatenamos todos los nombres de columnas ---
    all_features = np.concatenate([
        bin_features,
        ord_features,
        multi_features,
        scaling_features,
        remainder_features
    ])
    
    df_trans = pd.DataFrame(df_trans, columns=all_features)
    
    return df_trans


In [ ]:
pipe = create_preprocessing_pipeline(df)

In [ ]:
df_trans = reconstruct_transformed_df(pipe , df)
print(df_trans.info())
df_trans.head()

In [ ]:
create_churn_focused_heatmap(df_trans)

---